
# Local


In [2]:
! pip install --quiet -r ../requirements.txt

import sys
import os
sys.path.append(os.path.expanduser("~/repos/NeurOps/pytorch"))


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


# Colab


In [3]:
# ! git clone https://github.com/SuReLI/NeurOps.git
# ! git clone https://github.com/k8lion/GrowAndPrune.git
# %cd GrowAndPrune/src
# ! pip install --quiet -r ../requirements.txt

# import sys
# import os
# sys.path.append("../../NeurOps/pytorch")

# Imports

In [4]:
import numpy as np
import torch
import copy
import math
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML


from neurops import *

from growprune import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/kaitlin/repos/GrowAndPrune/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kaitlin/repos/GrowAndPrune/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
torch.load("https://download.pytorch.org/models/vgg11-8a719046.pth")

FileNotFoundError: [Errno 2] No such file or directory: 'https://download.pytorch.org/models/vgg11-8a719046.pth'